### Import library

Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP
    Goal is to predict if mutation is SNP or PD
    XG Boost
        
    Total samples: 3368
    2254 PD samples
    1111 SNP samples
    3 NA samples

MCC eval metric branch

In [43]:
""" Imports the required libraries and packages """

import pandas as pd                                                              # Data manipulation in dataframes
import numpy as np                                                               # Array manipulation
import xgboost as xgb                                                            # Gradient boosting package

import random as rd                                                              # Random seed generation
import time                                                                      # Time program run time
import sys

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from sklearn.metrics import(
    matthews_corrcoef,                                                           # MCC for evaluation
    # balanced_accuracy_score, #hyperparameter evaluation
    # f1_score,  #hyperparameter evaluation
    confusion_matrix,                                                            # Confusion matrix for classification evalutation
    classification_report                                                        # Return the F1, precision, and recall of a prediction
    )

from sklearn.model_selection import(
    train_test_split,                                                            # Splits data frame into the training set and testing set
    # GridSearchCV,  # Searches all hyperparameters
    # RandomizedSearchCV, # Searches random range of hyperparameters
    GroupKFold                                                                   # K-fold CV with as groups
        )

from sklearn.utils import shuffle
# from sklearn.ensemble import RandomForestClassifier                              # SK learn API for classificastion random forests

np.set_printoptions(precision = 3,threshold=np.inf, suppress=True)               # Full array printing

### Split dataset into training and validation sets

In [44]:
def Train_Test_Split(file):
    """      
    Input:      file             Pre-processed dataset done by PDB2AC script

    Returns:    Training_Set     80% training set split
                Testing_Set      20% testing set split
                
    80% training and 20% testing split. Splits are shuffled randomly and index reset
    """
    AC_dataset                  = pd.read_csv(file, index_col=0)  
    Training_Set, Testing_Set   = train_test_split(AC_dataset,train_size = 0.8)
        
    Training_Set                = Training_Set.sample(frac = 1) #Shuffle data after splitting
    Testing_Set                 = Testing_Set.sample(frac = 1)
    
    Training_Set.reset_index(drop=True, inplace = True) #Drop index to avoid training on index values
    Testing_Set.reset_index(drop=True, inplace = True)  #Reset index after splitting for compatability with group fold CV
    
    return Training_Set, Testing_Set

In [45]:
def data(Training_Set, Testing_Set):
    """      
    Input:      Training_Set     80% training set split
                Testing_Set      20% testing set split

    Returns:    train_features   Features for training
                train labels     Class lables for training
                test_features    Features for testing
                test_labels      Class labels for testing
                
    Creates the datasets needed for GBC model training and predictions
    """
    
    train_features     = Training_Set.drop(['AC Code','dataset'], axis =1)      
    train_labels       = Training_Set['dataset']                                  
        
    test_features     = Testing_Set.drop(['AC Code','dataset'], axis =1)         
    test_labels       = Testing_Set['dataset']                                  
        
    return(train_features, train_labels, test_features, test_labels)

### Initial evaluation

In [46]:
# def test(inData, classData, ValData, Vallabel):
#     """ 
#     Input:      inData          Features for training
#                 classData       Class lables for training
#                 valData         Features for testing
#                 Vallabel        Class labels for testing

#     Evaluate training data before CV and balancing. Gradient boosting for prediction on the test data. 
#     True values are testing data class labels
#     """    
#     d_train = xgb.DMatrix(inData, classData)
#     d_test = xgb.DMatrix(ValData, Vallabel)

#     params = {
#     'booster': 'gbtree',
#     'objective': 'binary:hinge', 
#     }
#     XGB_initial = xgb.train(params, d_train)
    
#     Output_pred = XGB_initial.predict(d_test)
#     print(f"              **Initial Evaluation**")
#     print(f"Confusion Matrix:\n {confusion_matrix(Vallabel, Output_pred)}")
#     print(f"MCC              {matthews_corrcoef(Vallabel, Output_pred)}\n")


## Group K-fold CV (outer loop)

In [47]:
def CV(Training_Set):
    """      
    Input:      Training_Set     80% training set split
            
    Returns:    IT_list         List of training features for each fold
                LT_list         List of training class labels for each fold
                IV_list         List of validation features for each fold
                LV_list         List of validation class labels for each fold

    K-fold CV with protein groups separated between training and validation sets for each fold. Creates 5 folds.
    """
    
    features     = Training_Set.drop(['AC Code','dataset'], axis =1)         #Features for training
    labels       = Training_Set['dataset']                         #Class labels for training
    groups       = Training_Set['AC Code'].to_list()               #List of proteins for grouping
    
    IT_list = []
    LT_list = []
    IV_list = []
    LV_list = []
    
    CV             = GroupKFold(n_splits = 5)                           #Creates 5 splits

    for train_idx, val_idx in CV.split(features, labels, groups):       #Generates the indices to be used for a training and validation split. Indicies are unique to train/ val sets

        Input_train                        = features.iloc[train_idx].reset_index(drop = True)   #New dataframe from selected indices
        Classes_train                      = labels.iloc[train_idx].reset_index(drop = True)
                
        Input_val                          = features.iloc[val_idx].reset_index(drop = True)
        Classes_val                        = labels.iloc[val_idx].reset_index(drop = True)

        IT_list.append(Input_train)          #Shuffles lists, random state to ensure features and labels match for each fold
        LT_list.append(Classes_train)
        IV_list.append(Input_val)
        LV_list.append(Classes_val)
    
    # params = {
    # 'booster': 'gbtree',
    # 'objective': 'binary:logistic', 
    # }
    
    # d_train = xgb.DMatrix(features, labels)
    # xgb_model = xgb.train(params, d_train)
        
    # CV = xgb.cv(
    #     params=params,
    #     dtrain = d_train,
    #     nfold = 5,
    #     early_stopping_rounds= 20,
    #     metrics='error',
    #     as_pandas=True,
    # )
    
    # print(CV)
        
    # CV             = GroupKFold(n_splits = 5)                           #Creates 5 splits
    
  
    
    # for train_idx, val_idx in CV.split(Input_CV, Output_CV, Protein_Groups): #Generates the indices to be used for a training and validation split. Indicies are unique to train/ val sets
        
    #     Rd = np.random.randint(time.time())                                  #Random number from 1 to time since epoch

    #     Input_train                        = Input_CV.loc[train_idx]         #New dataframe from selected indices
    #     Classes_train                      = Output_CV.loc[train_idx]
    #     Input_train.drop(['AC Code'], axis = 1, inplace = True)              #Group identifer not needed for training
                
    #     Input_val                          = Input_CV.loc[val_idx]
    #     Classes_val                        = Output_CV.loc[val_idx]
    #     Input_val.drop(['AC Code'], axis   = 1, inplace = True)

    #     IT_list.append(Input_train.sample(frac=1, random_state=Rd))          #Shuffles lists, random state to ensure features and labels match for each fold
    #     LT_list.append(Classes_train.sample(frac=1, random_state=Rd))
    #     IV_list.append(Input_val.sample(frac=1, random_state=(Rd-1)))
    #     LV_list.append(Classes_val.sample(frac=1, random_state=(Rd-1)))
        

    return(IT_list, LT_list, IV_list, LV_list)


## Balancing (inner loop)

In [48]:
def find_minority_class(classData):
    """ 
    Input:        classData  Array of class labels

    Returns:      minClass   The label for the minority class
                  minSize    The number of items in the minority class
                  maxSize    The number of items in the majority class

    Find information about class size imbalance
    """
    
    Minority_count = 0
    Majority_count = 0
    for datum in classData:
        if datum == 1:
            Majority_count += 1
        elif datum == 0:
            Minority_count += 1

    minClass = 0
    minSize  = Minority_count
    maxSize  = Majority_count
    if Minority_count > Majority_count:
        minClass = 1
        minSize  = Majority_count
        maxSize  = Minority_count

    return minClass, minSize, maxSize

In [49]:
def balance(inData, classData, minClass, minSize):
    """ 
    Input:        inData          array of input data
                  classData       array of classes assigned
                  minorityClass   class label for the minority class
                  minoritySize    size of the minority class

    Returns:      usedLines       array of indexes that are of interest for a balanced dataset

    Perform the actual balancing for a fold between SNPs and PDs
    """
    usedLines = [False] * len(inData) #Array of false for length of data
    for i in range(len(inData)):
        if classData[i] == minClass:
            usedLines[i] = True            #True lines are SNP
            
    usedCount = 0
    while usedCount < minSize:
        i = rd.randrange(len(inData))
        if usedLines[i] == False:
            usedLines[i] = True
            usedCount += 1          #Set PD lines "True", until equal to number of SNP lines

    return usedLines

In [50]:
def balance_data(inData, classData, usedLines):
    """     
    Input:      inData      array of input training data
                classData   array of classes assigned to training data
                usedLines   array of line indexes to print

    Returns:    input_balance  Dataframe of balanced training features
                label_balance  Dataframe of balanced training labels
                       
    Create dataframe of the input training data and classes used. Index_list preserves the indicies between usedLines and inData, used to pull the needed lines.
    """
    Rd = np.random.randint(time.time())
    index_list = []
    
    for i in range(len(usedLines)):
        if usedLines[i] == True:
            index_list.append(i)
             
    input_balance = inData.iloc[index_list] 
    label_balance = classData.iloc[index_list]   
    
    input_balance = input_balance.sample(frac=1, random_state=Rd).reset_index(inplace = False, drop = True)
    label_balance = label_balance.sample(frac=1, random_state=Rd).reset_index(inplace = False, drop = True)
    
    return input_balance, label_balance

In [51]:
def Balance_ratio(maxSize, minSize): 
    """ 
    Input:      maxSize     The number of items in the majority class
                minSize     The number of items in the minority class

    Returns:    BF          Number of balancing folds

    Calculate the number of balancing folds needed using ratio of majority to minority class size. Double to ensure sufficient
    majority class instances are sampled, then + 1 to make odd to allow weighted vote.
    """
    Divide = maxSize/minSize
    BF = (2 * round(Divide)) + 1 #Double ratio to nearest integer
    return BF

In [52]:
def Balance_Folds(BF, inData, classData, minClass, minSize):
    """ 
    Input:      BF                Number of balancing folds
                inData            Features for training
                classData         Class labels for training
                minClass          The label for the minority class
                minSize           The number of items in the minority class
                                  
    Returns:    Input_folds       List of balanced training feature folds
                Output_folds      List of balanced training label folds

    Perform the balance_data() function n number of balancing fold times. Return lists for feature data and labels
    where each item is the output of balance_data()
    """
    Input_folds  = []
    Output_folds = []

    for i in range(BF):
        usedLines                    = balance(inData, classData, minClass, minSize)
        input_balance, label_balance = balance_data(inData, classData, usedLines)
        
        Input_folds.append(input_balance)
        Output_folds.append(label_balance)
            
    return Input_folds, Output_folds

### XGB hyperparameter tuning

In [53]:
# def Hyperparameter(BF, Input_folds, Output_folds):
#     """ Input:      BF                Number of balancing folds needed
#                     Input_folds       List of 5 balanced arrays of training data
#                     Output_folds      List of 5 balanced arrays of training data's labels

#         Returns:    BF_RFC_HP         List of optimized hyperparameters for each RFC

#         Perform RandomSearchCV on each RFC to optimize number of trees, max depth and max samples
#     """  
#     estimator = RandomForestClassifier()
#     param_grid = {
#                 'n_estimators':np.arange(50,500,50),
#                 'max_depth': np.arange(2, 10, 2),
#                 'max_samples': np.arange(0.2, 1.2, 0.2)
#                   }
#     BF_RFC_HP = []

#     for i in range(BF):
#         HPtuning = RandomizedSearchCV(
#             estimator,
#             param_grid, 
#             scoring = 'balanced_accuracy',
#             cv = 10,
#             n_jobs = 6, #how many cores to run in parallel
#             verbose = 2
#             ).fit(Input_folds[i], Output_folds[i])
#         BF_RFC_HP.append(HPtuning.best_params_)
    
#     return(BF_RFC_HP)

### Train XGB on the trainings folds

In [54]:
def model_training_data(BF, Input_folds, Output_folds, ValData, Vallabel):
    """ 
    Input:      BF                Number of balancing folds
                Input_folds       List of balanced training feature folds
                Output_folds      List of balanced training label folds
                ValData           Unseen validation features from CV fold
                ValLabel          Unseen valiadation labels from CV fold
                                  
    Returns:    d_train_list      List of balanced training feature folds in DMatrix
                d_test            Validation data as Dmatrix

    Converts the balanced and validation data into Dmatrix for model training and evaluation
    """

    d_train_list =[]
    for i in range(BF):
            d_train = xgb.DMatrix(Input_folds[i], Output_folds[i])      #Create DMatrix for each training balanced fold
            d_train_list.append(d_train)
    d_test = xgb.DMatrix(ValData, Vallabel)

    return (d_train_list, d_test)


In [55]:
def CM(pred, d_test):
    """ 
    Input:      pred              Prediction from a boosted tree during training
                d_test            Validation data as Dmatrix

    MCC as a custom evaluation metric for evaluating the model during training. This is different from the final weighted evaluation
    """
    true_label = d_test.get_label()   
    pred_label = np.round(pred) 
    
    # CM = confusion_matrix(true_label, pred_label)
    # error = (CM[0, 1] + CM[1,0])/(CM[0, 1] + CM[1,0] + CM[1, 1] + CM[0,0])

    
    return 'mcc', matthews_corrcoef(pred_label, true_label)

In [56]:
def BF_fitting(BF, d_train_list, d_test): 
    """ 
    Input:      BF                Number of balancing folds                      
                d_train_list      List of balanced training feature folds in DMatrix
                d_test            Validation data as Dmatrix
                
    Returns:    BF_GBC            List of gradient boosted trees trained on each balancing fold

    Create GBC model that returns probability predictions for each fold, using output of Balance_Folds() as training data (as a Dmatrix)
    """     
    params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic', 
    'disable_default_eval_metric': 1,
    'verbosity': 1,
    } 
    
    BF_GBC = []
    for fold_i in range(BF):
        d_train = d_train_list[fold_i]                          #Dmatrix for each balanced fold
        BF_GBC.append(xgb.train(params, 
                                d_train, 
                                num_boost_round = 1000,
                                evals  = [(d_test,'Model')],
                                verbose_eval = 10,               #Print evaluation metrics every 5 trees
                                early_stopping_rounds = 50,
                                custom_metric = CM, 
                                )
                      )                                         #Generates and fits a GBC for each training balanced fold
        
        
    return BF_GBC

#### Validate each GBC on validation set, for each fold

In [57]:
def BF_validate(BF_GBC, d_test):
    """ 
    Input:      BF_RFC            List of RFCs trained on balancing folds
                d_test            Validation data as Dmatrix

                
    Returns:    Prob_matrix     List of arrays. Each item is 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability
    
    Test the trained RFCs on the test set, then for every instance, outputs the predicted probability for each class
    """
    
    Prob_matrix = []
    for i in range(len(BF_GBC)):
        Prob = BF_GBC[i].predict(d_test) #Predicts the probability of an instance belonging to the major/ positive class (PD/ 1). Output has shape (n_predictions,)
        Prob_matrix.append(Prob)   
        
    return Prob_matrix

### Weighted voting

In [58]:
def Weighted_Vote(Prob_matrix):
    """ 
    Input:      Prob_matrix     List of arrays. 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability

    Returns:    Final_vote      Weighted vote classification

    Calculate the final weighted vote using confidence scores (Sc) from Prob_matrix. Binary classification formula for:
    Predictor states its prediction/ confidence scores are between 0.0 and 1.0 for each class
    """
    PD_prob_matrix = Prob_matrix 
    
    SNP_prob_matrix = []
    for i in range(len(Prob_matrix)):               #SNP probabilites are 1 - (PD probabilites)
        sub = 1 - Prob_matrix[i]
        SNP_prob_matrix.append(sub)
            
    Sum_SNP = np.sum(SNP_prob_matrix, axis = 0)     #Sum of all SNP confidence scores. 1D Array
    Sum_PD  = np.sum(PD_prob_matrix, axis = 0)      #Sum of all PD confidence scores. 1D Array
                                                    
    Vote_arr  = [] 

    for i in range(len(Sum_PD)):
        if Sum_PD[i] >= Sum_SNP[i]:
            Vote_arr.append([1])                #Append PD classifications to list
        elif Sum_SNP[i] > Sum_PD[i]:
            Vote_arr.append([0])                #Append SNP classifications to list

    Final_vote = np.stack(Vote_arr)             #Converts list of arrays to a 2D array
    Final_vote = Final_vote.ravel()             #Flattens 2D array to 1D array

    return(Final_vote, Sum_PD, Sum_SNP)         #Returns the final confidence scores


### Evaluations

In [59]:
def evalutation(Final_vote, Vallabel):
    """ 
    Input:      Final Vote      Weighted vote classification
                ValLabel        Unseen 20% testing data labels

    Evaluate each fold with confusion matrix and MCC
    """

    Output_pred = Final_vote
    MCC = matthews_corrcoef(Vallabel, Output_pred)
    print(f"-----------------------------------------------------\n              ***Fold Evaluation***\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Vallabel, Output_pred)}")
    print(f"{classification_report(Vallabel, Output_pred)}\nMCC                           {MCC}\n")
    
    return MCC

In [60]:
def plot(Score_list):
     """ 
     Input:      Score_list        List of MCC scores

     Plots the MCCs of n runs, and calculates the average MCC
     """
     fig, ax = plt.subplots(figsize=(16,10), dpi= 65)
     x_axis = range(len(Score_list))
     y_axis = Score_list

     plt.scatter(x_axis, y_axis, color = 'teal')
     plt.axhline(y=np.nanmean(Score_list), color = 'red', linestyle = 'dotted', linewidth = '1', label ='Avg')
     plt.title('MCC of 15 XG Boost runs, no CV')
     plt.xlabel('Run Number')
     plt.ylabel('MCC')
     plt.legend()
     plt.show()
     print(f"Average: {np.nanmean(Score_list)}")

### Main Program

In [61]:
file = "AC_dataset.csv"
Score_list = []
# for i in range(0,5):
Training_Set, Testing_Set          = Train_Test_Split(file)                                 #Create training and testing sets
    # test(inData, classData, ValData, Vallabel)                                                          #Initial evaluation
IT_list, LT_list, IV_list, LV_list = CV(Training_Set)                #Cross-validate training set        

for i in range(len(IT_list)):          
    inData = IT_list[i]
    classData = LT_list[i]
    ValData = IV_list[i]
    Vallabel = LV_list[i]

    minClass, minSize, maxSize  = find_minority_class(classData)                            #Determines imbalance
    BF                          = Balance_ratio(maxSize, minSize)                           #Determins number of balancing folds needed
    Input_folds, Output_folds   = Balance_Folds(BF, inData, classData, minClass, minSize)   # balance() and balance_data() functions are called under this
    d_train_list, d_test        = model_training_data(BF, Input_folds, Output_folds, ValData, Vallabel)
    # BF_RFC_HP = Hyperparameter(BF, Input_folds, Output_folds)
    BF_GBC                      = BF_fitting(BF, d_train_list, d_test)
    Prob_matrix                 = BF_validate(BF_GBC, d_test)

    Final_vote, Sum_PD, Sum_SNP = Weighted_Vote(Prob_matrix)

    MCC = evalutation(Final_vote, Vallabel)
    Score_list.append(MCC)  
      
# plot(Score_list)

[0]	Model-mcc:0.28953
[10]	Model-mcc:0.31404
[20]	Model-mcc:0.36391
[30]	Model-mcc:0.29582
[40]	Model-mcc:0.29631
[50]	Model-mcc:0.28237
[0]	Model-mcc:0.28323
[10]	Model-mcc:0.31128
[20]	Model-mcc:0.32773
[30]	Model-mcc:0.28740
[40]	Model-mcc:0.29582
[50]	Model-mcc:0.30082
[60]	Model-mcc:0.29285
[70]	Model-mcc:0.29285
[80]	Model-mcc:0.28237
[90]	Model-mcc:0.28740
[100]	Model-mcc:0.28740
[110]	Model-mcc:0.28992
[120]	Model-mcc:0.29747
[123]	Model-mcc:0.29205
[0]	Model-mcc:0.24219
[10]	Model-mcc:0.29171
[20]	Model-mcc:0.30136
[30]	Model-mcc:0.26858
[40]	Model-mcc:0.29205
[50]	Model-mcc:0.30792
[0]	Model-mcc:0.31948
[10]	Model-mcc:0.36871
[20]	Model-mcc:0.39230
[30]	Model-mcc:0.38218
[40]	Model-mcc:0.37403
[49]	Model-mcc:0.39004
[0]	Model-mcc:0.39230
[10]	Model-mcc:0.44442
[20]	Model-mcc:0.39843
[30]	Model-mcc:0.34759
[40]	Model-mcc:0.35801
[50]	Model-mcc:0.34759
[60]	Model-mcc:0.34262
[70]	Model-mcc:0.35603
[80]	Model-mcc:0.34369
[83]	Model-mcc:0.34615
-----------------------------------